# Anomaly Score comparison

In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)
import torch
from torchvision import datasets

from anomaly_scores.energy import energy_anomaly_score
from anomaly_scores.max_logit import max_logit_anomaly_score
from anomaly_scores.softmax import max_softmax_anomaly_score
from anomaly_scores.vim_scores import VIM
from energy_ood.CIFAR.models.wrn import WideResNet
from energy_ood.utils.svhn_loader import SVHN
from energy_ood.utils.tinyimages_80mn_loader import TinyImages
from util import TEST_TRANSFORM, TINY_TRANSFORM
from util.display_results import compare_all_results
from util.get_ood_score import get_ood_score_for_multiple_datasets
from vim_training.test import test

## The data
Let's start with replicating the results from the paper. First, with the SVHN data set. 

In [2]:
loaders = []

id_data = datasets.CIFAR10("../data/cifar10", train=False, transform=TEST_TRANSFORM)
id_loader = torch.utils.data.DataLoader(
    id_data, batch_size=200, shuffle=False, num_workers=2, pin_memory=True
)
loaders.append(("CIFAR10", id_loader))


ood_data = SVHN(
    root="../data/svhn/",
    split="test",
    transform=TEST_TRANSFORM,
    download=False,
)
ood_loader = torch.utils.data.DataLoader(
    ood_data, batch_size=200, shuffle=True, num_workers=2, pin_memory=True
)
ood_num_examples = len(loaders[0][1].dataset) // 5
loaders.append(("SVHN", ood_loader))


data = datasets.CIFAR100("../data/cifar-100", train=False, transform=TEST_TRANSFORM)
loader = torch.utils.data.DataLoader(
    data, batch_size=200, shuffle=True, num_workers=2, pin_memory=True
)
loaders.append(("CIFAR100", loader))

## Models

We are using the Wide ResNet as in the paper.

In [3]:
models = []
model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(
    torch.load(
        "../energy_ood/CIFAR/snapshots/pretrained/cifar10_wrn_pretrained_epoch_99.pt"
    )
)
model.eval()
_ = model.cuda()
models.append(("WRN", model))



model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(
    torch.load(
        "../energy_ood/CIFAR/snapshots/energy_ft/cifar10_wrn_s1_energy_ft_epoch_9.pt"
    )
)
model.eval()
_ = model.cuda()
models.append(("Energy_ft", model))


model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(torch.load("../snapshots/energy/CIFAR10_WRN_epoch_9.pt"))
model.eval()
_ = model.cuda()
models.append(("Energy_ft_ours", model))

model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(torch.load("../snapshots/vanilla_ft/CIFAR10_WRN_epoch_9.pt"))
model.eval()
_ = model.cuda()
models.append(("vanilla_ft", model))

# Anomaly Scores
Let's compare the scores.

In [4]:
all_anomaly_results = {}
for model_name, model in models:
    print(model_name)
    all_anomaly_results[model_name] = {}
    vim = VIM(id_loader, model)

    scores = [
        ("MaxLogit", max_logit_anomaly_score),
        ("MaxSoftmax", max_softmax_anomaly_score),
        ("Energy", energy_anomaly_score),
        ("VIM", vim.compute_anomaly_score),
    ]

    _, test_accuracy = test(model, loaders[0][1])
    all_anomaly_results[model_name]["test_acc"] = test_accuracy

    for name, score in scores:
        print("  ", name)
        results = get_ood_score_for_multiple_datasets(
            loaders,
            model,
            score,
            is_using="last" if not name == "VIM" else "last_penultimate",
            runs=3,
        )
        aurocs = [np.mean(result[0]) for result in results]
        aurocs.append(np.mean(aurocs))
        all_anomaly_results[model_name][name] = aurocs

WRN
   MaxLogit
   MaxSoftmax
   Energy
   VIM
Energy_ft
   MaxLogit
   MaxSoftmax
   Energy
   VIM
Energy_ft_ours
   MaxLogit
   MaxSoftmax
   Energy
   VIM
vanilla_ft
   MaxLogit
   MaxSoftmax
   Energy
   VIM


In [5]:
compare_all_results(all_anomaly_results, loaders)

              WRN (5.15%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    91.09%    |    87.21%    |    89.15%   
               MaxSoftmax |    92.14%    |   *88.26%    |    90.20%   
                   Energy |    90.93%    |    86.75%    |    88.84%   
                      VIM |   *94.94%    |    85.72%    |   *90.33%   

        Energy_ft (4.85%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    99.13%    |    93.49%    |    96.31%   
               MaxSoftmax |    97.61%    |    92.20%    |    94.90%   
                   Energy |   *99.13%    |   *93.56%    |   *96.35%   
                      VIM |    64.87%    |    72.76%    |    68.82%   

   Energy_ft_ours (4.88%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |   *99.28%    |   *93.75%    |   *96.51%   
               MaxSoftmax |    96.85%    |    92.16%    |    94.51%   
                   Energy |    99.27%    |    93.63%    |    96.45%   
    